In [3]:
import pandas as pd

merged_data = pd.read_csv('merged_data.csv')

In [4]:
# Summary statistics for all Farm Boy locations
print(merged_data[['Median age of the population', 'Population, 2021', 
                   'population_density', 'LANDAREA']].describe())

# Or get the mean values
avg_demographics = merged_data[['Median age of the population', 
                                'Population, 2021', 
                                'population_density']].mean()
print("\nAverage Farm Boy Market Profile:")
print(avg_demographics)

       Median age of the population  Population, 2021  population_density  \
count                     51.000000         51.000000           51.000000   
mean                      40.231373      35496.333333         3899.552618   
std                        3.992067      19009.449041         5765.050443   
min                       31.400000      10332.000000          235.662380   
25%                       38.000000      19499.000000         1304.999021   
50%                       40.000000      32046.000000         2004.539154   
75%                       42.200000      43894.500000         3177.601235   
max                       48.000000     100835.000000        31244.619799   

         LANDAREA  
count   51.000000  
mean    25.962594  
std     32.727171  
min      0.495100  
25%      6.728700  
50%     13.420400  
75%     33.340000  
max    182.158900  

Average Farm Boy Market Profile:
Median age of the population       40.231373
Population, 2021                35496.333333
po

In [5]:
# Group stores by population density ranges
merged_data['density_category'] = pd.cut(merged_data['population_density'], 
                                          bins=[0, 50, 500, 2000, 20000],
                                          labels=['Rural', 'Suburban', 'Urban', 'Dense Urban'])

print(merged_data.groupby('density_category').size())

density_category
Suburban        3
Urban          22
Dense Urban    24
dtype: int64


In [6]:
# Distribution of median age
print(f"Average median age: {merged_data['Median age of the population'].mean():.1f}")
print(f"Age range: {merged_data['Median age of the population'].min():.1f} - {merged_data['Median age of the population'].max():.1f}")

Average median age: 40.2
Age range: 31.4 - 48.0


In [7]:
# Population served
print(f"Total population served: {merged_data['Population, 2021'].sum():,.0f}")
print(f"Average population per FSA: {merged_data['Population, 2021'].mean():,.0f}")

Total population served: 1,810,313
Average population per FSA: 35,496


In [8]:
# Typical Farm Boy market profile
profile = {
    'Avg Population': merged_data['Population, 2021'].mean(),
    'Avg Population Density': merged_data['population_density'].mean(),
    'Avg Median Age': merged_data['Median age of the population'].mean(),
    'Most Common Density Type': merged_data['density_category'].mode()[0]
}

print("\nFarm Boy Typical Market Profile:")
for key, value in profile.items():
    print(f"{key}: {value}")


Farm Boy Typical Market Profile:
Avg Population: 35496.333333333336
Avg Population Density: 3899.55261762151
Avg Median Age: 40.23137254901961
Most Common Density Type: Dense Urban
